# __HyDE-Q Pipe-line__

In [1]:
import os
import time
import pickle
import random
import faiss
import pandas as pd
import numpy as np
from numpy import dot
from numpy.linalg import norm
from tqdm import tqdm
from konlpy.tag import Kkma
from datasets import load_dataset
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForCausalLM

from HyDE import HyDEQ
from Promptor import Promptor
from Searcher import FaissSearcher
from Generator import Generator

#### ① 데이터셋 로드

In [7]:
# train set과 validation set의 context를 모두 사용
dataset = load_dataset('squad_kor_v1')
contexts = dataset['validation']['context']

In [12]:
len(list(set(contexts)))

960

#### ② HyDE-Q Pipe Line 선언

In [13]:
# 검색모델(unsupervised)과 생성모델 로드
# encoder = AutoModel.from_pretrained('snunlp/KR-SBERT-V40K-klueNLI-augSTS')
encoder = SentenceTransformer.from_pretrained('snunlp/KR-SBERT-V40K-klueNLI-augSTS')
model = AutoModelForCausalLM.from_pretrained('beomi/KoAlpaca-Polyglot-5.8B')
tokenizer = AutoTokenizer.from_pretrained('beomi/KoAlpaca-Polyglot-5.8B')
tokenizer.pad_token = tokenizer.eos_token
model.resize_token_embeddings(len(tokenizer))

Loading checkpoint shards:   0%|          | 0/13 [00:00<?, ?it/s]

Embedding(30003, 4096)

In [51]:
# 프롬프터, 생성모델, 임베딩 모델, 검색도구를 선언
promptor = Promptor('koquad')
generator = Generator(model, tokenizer)
searcher = FaissSearcher(encoder, contexts, 'koquad_sbert.index')

Load Faiss from disc


In [52]:
# HyDEQ의 Pipe Line
hydeq = HyDEQ(promptor, generator, encoder, searcher.index)

#### ③ 질의로부터 가짜 답안(Hypothesis) 생성

In [16]:
# 쿼리와 프롬프트 확인
query = '전통 신학계의 근본주의적 시작을 여전히 받아들여 노아의 방주를 역사적 사실로 인식하는 집단은?'
prompt = hydeq.prompt(query)
prompt

'다음 질문에 대답할 수 있는 텍스트를 만들어주세요. ###질문: 전통 신학계의 근본주의적 시작을 여전히 받아들여 노아의 방주를 역사적 사실로 인식하는 집단은? ###답변:'

In [17]:
hypothesis_documents = hydeq.generate_hypothesis(query)
for i, doc in enumerate(hypothesis_documents):
    print(f'HyDE Generated Document: {i}')
    print(doc.strip())

/home/snumin44/myanaconda/envs/audio/lib/python3.8/site-packages/transformers/generation/utils.py:1473: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


HyDE Generated Document: 0
대한기독교대한성경신학대학원대학교의 교수님들과 학생들, 그리고 숭실대 한국기독교문화학부의 교수님들과 학생들은 그들의 역사적 배경과 인식을 근거로 하여, 노아의 방주가 역사적 사실이라고 주장합니다. 이들은 또한 예수 그리스도가 새로운 시대를 열 수 있도록 하나님이 주재하
HyDE Generated Document: 1
그렇습니다. 전통적인 신학계에서는 노아의 방주를 역사적 사실로 인지하고 있습니다. 이에 반해 세속화된 교회에서는 이에 관한 문제제기가 나오고 있으며, 이와 관련된 역사적인 사건들을 보는 관점에 대한 논의가 진행되고 있습니다.
HyDE Generated Document: 2
X파일에 속한 사람들은 신비주의적인 성향을 가지고 있어서 질문자가 제시한 텍스트를 역사적 사실로 인식하는데 어려움이 있을 수 있습니다. 하지만 이들은 여전히 노아의 방주를 전설적인 이야기로 인식하고 있습니다.


#### ④ 가짜 답안을 이용해 유사 문서 검색

In [18]:
# 가짜 답안을 인코딩
hyde_vector = hydeq.encode_documents(hypothesis_documents)
print(hyde_vector.shape)

(1, 768)


In [19]:
# 결과 확인 (정답은 2번째 문서)
hits = hydeq.search(hyde_vector, k=50)
hits = [{'index': id, 'score': score} for score, id in zip(hits[0][0], hits[1][0])]
hits = sorted(hits, key=lambda x: x['score'], reverse=False)

hits_lst = []
for id, hit in enumerate(hits):
  print(f'HyDE-Q Retrieved Document: {id}')
  print("\t{:.3f}\t{}".format(hit['score'], searcher.all_contexts[hit['index']]))
  hits_lst.append(searcher.all_contexts[hit['index']])

HyDE-Q Retrieved Document: 0
	86.106	물론 노아의 방주가 신학과 신앙에서 중요한 영향을 차지하는 것은 사실이나, 현재 노아의 방주가 역사적으로 실존한다는 주장은 그 증거가 존재하지 않기에 관련 학계로부터 전혀 인정받지 못하고 있으며 그 실존과 안정성에 대한 수많은 논란이 있다. 한국창조과학회 등에서는 제칠일안식교를 기반으로 한 홍수지질학적 주장들을을 내어 놓고 있지만, 사실과 다른 근거들을 바탕으로 주장하므로 신뢰하기 힘든 것들이 전부라 할 수 있다. 그러므로 현재 노아의 방주가 실존한다는 주장은 그 증거가 존재하지 않기에 관련 학계로부터 전혀 인정받지 못하고 있다. 모든 과학관련 학계에서는 노아의 방주의 구조나 재질등이 실제로 존재할 수 없는 설화속 이야기라는 데에 동의하고 있다.
HyDE-Q Retrieved Document: 1
	90.371	역사학과 과학이 발달하지 않았던 과거 전통 신학계에서는 근본주의적 시각을 받아들여 노아의 방주를 역사적 사실로 기술하려 했으며, 이러한 관점은 아직도 과학과 역사학에 어두운 보수적 근본주의계열의 개신교에서만 받아들여지고 있다. 하지만 역사학과 과학의 발달로 인해, 노아의 방주의 실존에 대한 의문이 제기가 되고, 세계적 홍수가 존재할 수 없음이 밝혀짐에 따라 현대 신학계에서는 비록 노아의 홍수가 과학적으로 실존하지는 않았지만 그 자체의 의미는 신학적으로 매우 중요하며, 이에 대한 해석은 다양하게 이루어지고 있으며, 대부분의 기독교(가톨릭, 개신교를 포함한 대부분)에서는 노아의 방주는 상징적 의미로 받아들여진다. 그러므로 과학과는 상관없이 신학적으로 노아의 방주 자체의 의미는 중요하게 해석된다고 한다
HyDE-Q Retrieved Document: 2
	90.979	역사학과 과학의 발달이 더뎠던 고대사회에서는, 성경이 단순한 교리적인 부분 뿐 아니라 역사책으로서의 권위도 높았기에 노아의 방주를 역사적인 존재로서 다루고 있었다. 이는 제칠일안식교에서 비롯된 의사과학의 한 종류인 유사지질학인 홍

#### ⑤ 쿼리를 이용해 Re-Rank

In [20]:
# line을 기준으로 re-rank (정답은 1번째 문서)
rerank_hits = hydeq.rerank_by_line(hits_lst, query)
rerank_hits_lst = []
for id, hit in enumerate(rerank_hits):
  print(f'HyDEQ Re-ranked Retrieved Document: {id}')
  print("\t{:.3f}\t{}".format(hit['score'], hit['text']))
  rerank_hits_lst.append(hit['text'])

HyDEQ Re-ranked Retrieved Document: 0
	0.710	역사학과 과학이 발달하지 않았던 과거 전통 신학계에서는 근본주의적 시각을 받아들여 노아의 방주를 역사적 사실로 기술하려 했으며, 이러한 관점은 아직도 과학과 역사학에 어두운 보수적 근본주의계열의 개신교에서만 받아들여지고 있다. 하지만 역사학과 과학의 발달로 인해, 노아의 방주의 실존에 대한 의문이 제기가 되고, 세계적 홍수가 존재할 수 없음이 밝혀짐에 따라 현대 신학계에서는 비록 노아의 홍수가 과학적으로 실존하지는 않았지만 그 자체의 의미는 신학적으로 매우 중요하며, 이에 대한 해석은 다양하게 이루어지고 있으며, 대부분의 기독교(가톨릭, 개신교를 포함한 대부분)에서는 노아의 방주는 상징적 의미로 받아들여진다. 그러므로 과학과는 상관없이 신학적으로 노아의 방주 자체의 의미는 중요하게 해석된다고 한다
HyDEQ Re-ranked Retrieved Document: 1
	0.630	프리스틀리는 워링턴 거주 시절에는 다른 일 때문에 신학 연구에 몰두하지 못하였으나, 리즈에 오면서 그는 신학 연구에 많은 시간을 투자하였고, 결과적으로 그의 신앙은 아리우스주의에서 유니테리언으로 정립되었다. 리즈에서 프리스틀리는 삼위일체와 예수의 선재성(先在性, 성자인 예수는 천지창조전부터 성부와 같이 존재했다는 교리)등을 완전히 부정하였고, 기독교 교리와 성서를 새롭게 해석하기 시작했다. 그는 오래전부터 써오던 신학 교육에 대한 책인 《자연과 계시 종교의 원리》(Institutes of Natural and Revealed Religion)를 출판하기 시작하였는데, 1772년에 1권이 출판되었고 마지막 3권은 1774년에 출판되었다. 그는 책에서 자연 종교, 계시의 진실성을 뒷받침하는 논거, 계시로부터 얻을 수 있는 진실 등을 논했다. 이 책은 약 반세기간의 자유주의 신학자들의 주장을 요약하였고 오랜 기간에 걸쳐 유니테리언의 대표적인 해설서가 되었다.
HyDEQ Re-ranked Retrieved Docum

#### ⑥ 가장 유사도가 큰 문서로 정답 생성

In [53]:
# 쿼리와 프롬프트
query = '전통 신학계의 근본주의적 시작을 여전히 받아들여 노아의 방주를 역사적 사실로 인식하는 집단은?'
prompt = promptor.build_prompt(query, rerank_hits[0]['text'])
prompt

'###질문: 다음 텍스트를 참고해서 답해주세요. 전통 신학계의 근본주의적 시작을 여전히 받아들여 노아의 방주를 역사적 사실로 인식하는 집단은? 텍스트:역사학과 과학이 발달하지 않았던 과거 전통 신학계에서는 근본주의적 시각을 받아들여 노아의 방주를 역사적 사실로 기술하려 했으며, 이러한 관점은 아직도 과학과 역사학에 어두운 보수적 근본주의계열의 개신교에서만 받아들여지고 있다. 하지만 역사학과 과학의 발달로 인해, 노아의 방주의 실존에 대한 의문이 제기가 되고, 세계적 홍수가 존재할 수 없음이 밝혀짐에 따라 현대 신학계에서는 비록 노아의 홍수가 과학적으로 실존하지는 않았지만 그 자체의 의미는 신학적으로 매우 중요하며, 이에 대한 해석은 다양하게 이루어지고 있으며, 대부분의 기독교(가톨릭, 개신교를 포함한 대부분)에서는 노아의 방주는 상징적 의미로 받아들여진다. 그러므로 과학과는 상관없이 신학적으로 노아의 방주 자체의 의미는 중요하게 해석된다고 한다 ###답변:'

In [55]:
prediction = hydeq.generate_answer(query, rerank_hits[0]['text'], n=1)
print(prediction[0])

전통


#### ⑥ 검색과 생성의 정확도 확인

In [98]:
# 각 샘플에 대해 정답지를 만들어 종합해 채점
re_answer = '역사학과 과학이 발달하지 않았던 과거 전통 신학계에서는 근본주의적 시각을 받아들여 노아의 방주를 역사적 사실로 기술하려 했으며, 이러한 관점은 아직도 과학과 역사학에 어두운 보수적 근본주의계열의 개신교에서만 받아들여지고 있다. 하지만 역사학과 과학의 발달로 인해, 노아의 방주의 실존에 대한 의문이 제기가 되고, 세계적 홍수가 존재할 수 없음이 밝혀짐에 따라 현대 신학계에서는 비록 노아의 홍수가 과학적으로 실존하지는 않았지만 그 자체의 의미는 신학적으로 매우 중요하며, 이에 대한 해석은 다양하게 이루어지고 있으며, 대부분의 기독교(가톨릭, 개신교를 포함한 대부분)에서는 노아의 방주는 상징적 의미로 받아들여진다. 그러므로 과학과는 상관없이 신학적으로 노아의 방주 자체의 의미는 중요하게 해석된다고 한다'
ge_answer = '보수적 근본주의계열의 개신교'

def create_answer_sheet(query, re_answer, ge_answer, prediction, hits_lst):
    answer_sheet = {
        'top_1': 'O' if re_answer in hits_lst[:1] else 'X',
        'top_5': 'O' if re_answer in hits_lst[:5] else 'X',
        'top_10': 'O' if re_answer in hits_lst[:10] else 'X',
        'top_30': 'O' if re_answer in hits_lst[:30] else 'X',
        'top_50': 'O' if re_answer in hits_lst[:50] else 'X',
        'query': query,
        're_answer': re_answer,
        'ge_answer': ge_answer,
        'predict': prediction
    }
    return answer_sheet

create_answer_sheet(query, re_answer, ge_answer, prediction, rerank_hits_lst)

{'top_1': 'O',
 'top_5': 'O',
 'top_10': 'O',
 'top_30': 'O',
 'top_50': 'O',
 'query': '전통 신학계의 근본주의적 시작을 여전히 받아들여 노아의 방주를 역사적 사실로 인식하는 집단은?',
 're_answer': '역사학과 과학이 발달하지 않았던 과거 전통 신학계에서는 근본주의적 시각을 받아들여 노아의 방주를 역사적 사실로 기술하려 했으며, 이러한 관점은 아직도 과학과 역사학에 어두운 보수적 근본주의계열의 개신교에서만 받아들여지고 있다. 하지만 역사학과 과학의 발달로 인해, 노아의 방주의 실존에 대한 의문이 제기가 되고, 세계적 홍수가 존재할 수 없음이 밝혀짐에 따라 현대 신학계에서는 비록 노아의 홍수가 과학적으로 실존하지는 않았지만 그 자체의 의미는 신학적으로 매우 중요하며, 이에 대한 해석은 다양하게 이루어지고 있으며, 대부분의 기독교(가톨릭, 개신교를 포함한 대부분)에서는 노아의 방주는 상징적 의미로 받아들여진다. 그러므로 과학과는 상관없이 신학적으로 노아의 방주 자체의 의미는 중요하게 해석된다고 한다',
 'ge_answer': '보수적 근본주의계열의 개신교',
 'predict': ['###질문: 다음 텍스트를 보고 답해주세요. 전통 신학계의 근본주의적 시작을 여전히 받아들여 노아의 방주를 역사적 사실로 인식하는 집단은? 텍스트: 역사학과 과학이 발달하지 않았던 과거 전통 신학계에서는 근본주의적 시각을 받아들여 노아의 방주를 역사적 사실로 기술하려 했으며, 이러한 관점은 아직도 과학과 역사학에 어두운 보수적 근본주의계열의 개신교에서만 받아들여지고 있다. 하지만 역사학과 과학의 발달로 인해, 노아의 방주의 실존에 대한 의문이 제기가 되고, 세계적 홍수가 존재할 수 없음이 밝혀짐에 따라 현대 신학계에서는 비록 노아의 홍수가 과학적으로 실존하지는 않았지만 그 자체의 의미는 신학적으로 매우 중요하며, 이에 대한 해석은 다양하게 이루어지고 있으며, 대부분의 기독교(가톨릭, 개신교를 포함한 대